Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# 01. Handwritten Digit Classification (MNIST) using ONNX Runtime on AzureML

This example shows how to deploy an image classification neural network using the Modified National Institute of Standards and Technology ([MNIST](http://yann.lecun.com/exdb/mnist/)) dataset and Open Neural Network eXchange format ([ONNX](http://aka.ms/onnxdocarticle)) on the Azure Machine Learning platform. MNIST is a popular dataset consisting of 70,000 grayscale images. Each image is a handwritten digit of 28x28 pixels, representing number from 0 to 9. This tutorial will show you how to deploy a MNIST model from the [ONNX model zoo](https://github.com/onnx/models), use it to make predictions using ONNX Runtime Inference, and deploy it as a web service in Azure.

Throughout this tutorial, we will be referring to ONNX, a neural network exchange format used to represent deep learning models. With ONNX, AI developers can more easily move models between state-of-the-art tools (CNTK, PyTorch, Caffe, MXNet, TensorFlow) and choose the combination that is best for them. ONNX is developed and supported by a community of partners including Microsoft AI, Facebook, and Amazon. For more information, explore the [ONNX website](http://onnx.ai) and [open source files](https://github.com/onnx).

[ONNX Runtime](https://aka.ms/onnxruntime) is the runtime engine that enables evaluation of trained machine learning (Traditional ML and Deep Learning) models with high performance and low resource utilization.

#### Tutorial Objectives:

1. Describe the MNIST dataset and pretrained Convolutional Neural Net ONNX model, stored in the ONNX model zoo.
2. Deploy and run the pretrained MNIST ONNX model on an Azure Machine Learning instance
3. Predict labels for test set data points in the cloud using ONNX Runtime and Azure ML

## Prerequisites

### 1. Install Azure ML SDK and create a new workspace
Please follow [00.configuration.ipynb](https://github.com/Azure/MachineLearningNotebooks/blob/master/00.configuration.ipynb) notebook.


### 2. Install additional packages needed for this Notebook
You need to install the popular plotting library `matplotlib` and the `onnx` library in the conda environment where Azure Maching Learning SDK is installed.

```sh
(myenv) $ pip install matplotlib onnx
```

### 3. Download sample data and pre-trained ONNX model from ONNX Model Zoo.

[Download the ONNX MNIST model and corresponding test data](https://www.cntk.ai/OnnxModels/mnist/opset_7/mnist.tar.gz) and place them in the same folder as this tutorial notebook. You can unzip the file through the following line of code.

```sh
(myenv) $ tar xvzf mnist.tar.gz
```

More information can be found about the ONNX MNIST model on [github](https://github.com/onnx/models/tree/master/mnist). For more information about the MNIST dataset, please visit [Yan LeCun's website](http://yann.lecun.com/exdb/mnist/).

## Load Azure ML workspace

We begin by instantiating a workspace object from the existing workspace created earlier in the configuration notebook.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep = '\n')

### Registering your model with Azure ML

In [ ]:
model_dir = "mnist" # replace this with the location of your model files

# leave as is if it's in the same folder as this notebook

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path = model_dir + "//model.onnx",
                       model_name = "mnist_1",
                       tags = {"onnx": "demo"},
                       description = "MNIST image classification CNN from ONNX Model Zoo",
                       workspace = ws)

### Optional: Displaying your registered models

This step is not required, so feel free to skip it.

In [ ]:
models = ws.models()
for m in models:
    print("Name:", m.name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)

### ONNX MNIST Model Methodology

The image classification model we are using is pre-trained using Microsoft's deep learning cognitive toolkit, [CNTK](https://github.com/Microsoft/CNTK), from the [ONNX model zoo](http://github.com/onnx/models). The model zoo has many other models that can be deployed on cloud providers like AzureML without any additional training. To ensure that our cloud deployed model works, we use testing data from the famous MNIST data set, provided as part of the [trained MNIST model](https://github.com/onnx/models/tree/master/mnist) in the ONNX model zoo.

***Input: Handwritten Images from MNIST Dataset***

***Task: Classify each MNIST image into an appropriate digit***

***Output: Digit prediction for input image***

Run the cell below to look at some of the sample images from the MNIST dataset that we used to train this ONNX model. Remember, once the application is deployed in Azure ML, you can use your own images as input for the model to classify!

In [ ]:
# for images and plots in this notebook
import matplotlib.pyplot as plt  
from IPython.display import Image

# display images inline
%matplotlib inline

In [ ]:
Image(url="http://3.bp.blogspot.com/_UpN7DfJA0j4/TJtUBWPk0SI/AAAAAAAAABY/oWPMtmqJn3k/s1600/mnist_originals.png", width=200, height=200)

## Deploy our model on Azure ML

We are now going to deploy our ONNX Model on AML with inference in ONNX Runtime. We begin by writing a score.py file, which will help us run the model in our Azure ML virtual machine (VM), and then specify our environment by writing a yml file.

You will also notice that we import the onnxruntime library to do runtime inference on our ONNX models (passing in input and evaluating out model's predicted output). More information on the API and commands can be found in the [ONNX Runtime documentation](https://aka.ms/onnxruntime).

### Write Score File

A score file is what tells our Azure cloud service what to do. After initializing our model using azureml.core.model, we start an ONNX Runtime inference session to evaluate the data passed in on our function calls.

In [ ]:
%%writefile score.py
import json
import numpy as np
import onnxruntime
import sys
import os
from azureml.core.model import Model
import time


def init():
    global session
    model = Model.get_model_path(model_name = 'mnist_1')
    session = onnxruntime.InferenceSession(model, None)
    
def run(input_data):
    '''Purpose: evaluate test input in Azure Cloud using onnxruntime.
        We will call the run function later from our Jupyter Notebook 
        so our azure service can evaluate our model input in the cloud. '''

    try:
        # load in our data, convert to readable format
        start = time.time()
        data = np.array(json.loads(input_data)['data']).astype('float32')

        r = session.run(["Plus214_Output_0"], {"Input3": data})[0]
        result = choose_class(r[0])
        end = time.time()
        result_dict = {"result": np.array(result).tolist(),
                      "time": np.array(end - start).tolist()}
    except Exception as e:
        result_dict = {"error": str(e)}
    
    return json.dumps(result_dict)

def choose_class(result_prob):
    """We use argmax to determine the right label to choose from our output, after calling softmax on the 10 numbers we receive"""
    return int(np.argmax(result_prob, axis=0))

### Write Environment File

This step creates a YAML file that specifies which dependencies we would like to see in our Linux Virtual Machine.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_pip_package("numpy")
myenv.add_pip_package("azureml-core")
myenv.add_pip_package("onnxruntime")


with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

### Create the Container Image

This step will likely take a few minutes.

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "test",
                                                  tags = {"demo": "onnx"} 
                                                 )


image = ContainerImage.create(name = "onnxtest",
                              # this is the model object
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

### Debugging

In case you need to debug your code, the next line of code accesses the log file.

In [ ]:
print(image.image_build_log_uri)

We're all set! Let's get our model chugging.

## Deploy the container image

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'demo': 'onnx'}, 
                                               description = 'ONNX for mnist model')

The following cell will likely take a few minutes to run as well.

In [ ]:
from azureml.core.webservice import Webservice

aci_service_name = 'onnx-demo-mnist'
print("Service", aci_service_name)

aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)

aci_service.wait_for_deployment(True)
print(aci_service.state)

In [ ]:
if aci_service.state != 'Healthy':
    # run this command for debugging.
    print(aci_service.get_logs())

    # If your deployment fails, make sure to delete your aci_service or rename your service before trying again!
    # aci_service.delete()

### Success!

If you've made it this far, you've deployed a working VM with a handwritten digit classifier running in the cloud using Azure ML. Congratulations!

Let's see how well our model deals with our test images.

# Testing and Evaluation

## Load Test Data

These are already in your directory from your ONNX model download (from the model zoo). If you didn't place your model and test data in the same directory as this notebook, edit the "model_dir" filename below.

In [ ]:
# to manipulate our arrays
import numpy as np 

# read in test data protobuf files included with the model
import onnx
from onnx import numpy_helper

# to use parsers to read in our model/data
import json
import os

test_inputs = []
test_outputs = []

# read in 3 testing images from .pb files
test_data_size = 3

for i in np.arange(test_data_size):
    input_test_data = os.path.join(model_dir, 'test_data_set_{0}'.format(i), 'input_0.pb')
    output_test_data = os.path.join(model_dir, 'test_data_set_{0}'.format(i), 'output_0.pb')
    
    # convert protobuf tensors to np arrays using the TensorProto reader from ONNX
    tensor = onnx.TensorProto()
    with open(input_test_data, 'rb') as f:
        tensor.ParseFromString(f.read())
    
    input_data = numpy_helper.to_array(tensor)
    test_inputs.append(input_data)
    
    with open(output_test_data, 'rb') as f:
        tensor.ParseFromString(f.read())
    
    output_data = numpy_helper.to_array(tensor)
    test_outputs.append(output_data)
    
if len(test_inputs) == test_data_size:
    print('Test data loaded successfully.')

### Show some sample images
We use `matplotlib` to plot 3 test images from the dataset.

In [ ]:
plt.figure(figsize = (16, 6))
for test_image in np.arange(3):
    plt.subplot(1, 15, test_image+1)
    plt.axhline('')
    plt.axvline('')
    plt.imshow(test_inputs[test_image].reshape(28, 28), cmap = plt.cm.Greys)
plt.show()

### Run evaluation / prediction

In [ ]:
plt.figure(figsize = (16, 6), frameon=False)
plt.subplot(1, 8, 1)

plt.text(x = 0, y = -30, s = "True Label: ", fontsize = 13, color = 'black')
plt.text(x = 0, y = -20, s = "Result: ", fontsize = 13, color = 'black')
plt.text(x = 0, y = -10, s = "Inference Time: ", fontsize = 13, color = 'black')
plt.text(x = 3, y = 14, s = "Model Input", fontsize = 12, color = 'black')
plt.text(x = 6, y = 18, s = "(28 x 28)", fontsize = 12, color = 'black')
plt.imshow(np.ones((28,28)), cmap=plt.cm.Greys)    


for i in np.arange(test_data_size):
    
    input_data = json.dumps({'data': test_inputs[i].tolist()})
    
    # predict using the deployed model
    r = json.loads(aci_service.run(input_data))
    
    if len(r) == 1:
        print(r['error'])
        break
        
    result = r['result']
    time_ms = np.round(r['time'] * 1000, 2)
    
    ground_truth = int(np.argmax(test_outputs[i]))
    
    # compare actual value vs. the predicted values:
    plt.subplot(1, 8, i+2)
    plt.axhline('')
    plt.axvline('')

    # use different color for misclassified sample
    font_color = 'red' if ground_truth != result else 'black'
    clr_map = plt.cm.gray if ground_truth != result else plt.cm.Greys

    # ground truth labels are in blue
    plt.text(x = 10, y = -30, s = ground_truth, fontsize = 18, color = 'blue')
    
    # predictions are in black if correct, red if incorrect
    plt.text(x = 10, y = -20, s = result, fontsize = 18, color = font_color)
    plt.text(x = 5, y = -10, s = str(time_ms) + ' ms', fontsize = 14, color = font_color)

    
    plt.imshow(test_inputs[i].reshape(28, 28), cmap = clr_map)

plt.show()

### Try classifying your own images!

Create your own 28 pixel by 28 pixel handwritten image and pass it into the model.

In [ ]:
# Preprocessing functions

def rgb2gray(rgb):
    """Convert the input image into grayscale"""
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def preprocess(img):
    """Resize input images and convert them to grayscale."""
    if img.shape[0] != 28:
        print("Input image size is not 28 * 28 pixels. Please resize and try again.")
    grayscale = rgb2gray(img)
    grayscale.resize((1, 1, 28, 28))
    return grayscale

In [ ]:
# Replace this string with your own path/test image
# Make sure the dimensions are 28 * 28 pixels

# Any PNG or JPG image file should work
# Make sure to include the entire path with // instead of /

# e.g. your_test_image = "C://Users//vinitra.swamy//Pictures//digit.png"

your_test_image = "<path to file>"

import matplotlib.image as mpimg

if your_test_image != "<path to file>":
    img = mpimg.imread(your_test_image)
    plt.subplot(1,3,1)
    plt.imshow(img, cmap = plt.cm.Greys)
    print("Old Dimensions: ", img.shape)
    img = preprocess(img)
    print("New Dimensions: ", img.shape)
else:
    img = None

In [ ]:
if img is None:
    print("Add the path for your image data.")
else:
    input_data = json.dumps({'data': img.tolist()})

    try:
        r = json.loads(aci_service.run(input_data))
        result = r['result']
        time_ms = np.round(r['time'] * 1000, 2)
    except Exception as e:
        print(str(e), r['error'])

    plt.figure(figsize = (16, 6))
    plt.subplot(1, 15,1)
    plt.axhline('')
    plt.axvline('')
    plt.text(x = -100, y = -20, s = "Model prediction: ", fontsize = 14)
    plt.text(x = -100, y = -10, s = "Inference time: ", fontsize = 14)
    plt.text(x = 0, y = -20, s = str(result), fontsize = 14)
    plt.text(x = 0, y = -10, s = str(time_ms) + " ms", fontsize = 14)
    plt.text(x = -100, y = 14, s = "Input image: ", fontsize = 14)
    plt.imshow(img.reshape(28, 28), cmap = plt.cm.gray)    

## Optional: How does our MNIST model work? 
#### A brief explanation of Convolutional Neural Networks

A [convolutional neural network](https://en.wikipedia.org/wiki/Convolutional_neural_network) (CNN, or ConvNet) is a type of [feed-forward](https://en.wikipedia.org/wiki/Feedforward_neural_network) artificial neural network made up of neurons that have learnable weights and biases. The CNNs take advantage of the spatial nature of the data. In nature, we perceive different objects by their shapes, size and colors. For example, objects in a natural scene are typically edges, corners/vertices (defined by two of more edges), color patches etc. These primitives are often identified using different detectors (e.g., edge detection, color detector) or combination of detectors interacting to facilitate image interpretation (object classification, region of interest detection, scene description etc.) in real world vision related tasks. These detectors are also known as filters. Convolution is a mathematical operator that takes an image and a filter as input and produces a filtered output (representing say edges, corners, or colors in the input image).  

Historically, these filters are a set of weights that were often hand crafted or modeled with mathematical functions (e.g., [Gaussian](https://en.wikipedia.org/wiki/Gaussian_filter) / [Laplacian](http://homepages.inf.ed.ac.uk/rbf/HIPR2/log.htm) / [Canny](https://en.wikipedia.org/wiki/Canny_edge_detector) filter).  The filter outputs are mapped through non-linear activation functions mimicking human brain cells called [neurons](https://en.wikipedia.org/wiki/Neuron). Popular deep CNNs or ConvNets (such as [AlexNet](https://en.wikipedia.org/wiki/AlexNet), [VGG](https://arxiv.org/abs/1409.1556), [Inception](http://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Szegedy_Going_Deeper_With_2015_CVPR_paper.pdf), [ResNet](https://arxiv.org/pdf/1512.03385v1.pdf)) that are used for various [computer vision](https://en.wikipedia.org/wiki/Computer_vision) tasks have many of these architectural primitives (inspired from biology).  

### Convolution Layer

A convolution layer is a set of filters. Each filter is defined by a weight (**W**) matrix, and  bias ($b$).

![](https://www.cntk.ai/jup/cntk103d_filterset_v2.png)

These filters are scanned across the image performing the dot product between the weights and corresponding input value ($x$). The bias value is added to the output of the dot product and the resulting sum is optionally mapped through an activation function. This process is illustrated in the following animation.

In [ ]:
Image(url="https://www.cntk.ai/jup/cntk103d_conv2d_final.gif", width= 200)

### Model Description

The MNIST model from the ONNX Model Zoo uses maxpooling to update the weights in its convolutions, summarized by the graphic below. You can see the entire workflow of our pre-trained model in the following image, with our input images and our output probabilities of each of our 10 labels. If you're interested in exploring the logic behind creating a Deep Learning model further, please look at the [training tutorial for our ONNX MNIST Convolutional Neural Network](https://github.com/Microsoft/CNTK/blob/master/Tutorials/CNTK_103D_MNIST_ConvolutionalNeuralNetwork.ipynb). 

#### Max-Pooling for Convolutional Neural Nets

![](http://www.cntk.ai/jup/c103d_max_pooling.gif)

#### Pre-Trained Model Architecture

![](http://www.cntk.ai/jup/conv103d_mnist-conv-mp.png)

### Try classifying your own images!

Create your own 28 pixel by 28 pixel handwritten image and pass it into the model.

In [ ]:
# Preprocessing functions

def rgb2gray(rgb):
    """Convert the input image into grayscale"""
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def preprocess(img):
    """Resize input images and convert them to grayscale."""
    if img.shape[0] != 28:
        print("Input image size is not 28 * 28 pixels. Please resize and try again.")
    grayscale = rgb2gray(img)
    grayscale.resize((1, 1, 28, 28))
    return grayscale

In [ ]:
# Replace this string with your own path/test image
# Make sure the dimensions are 28 * 28 pixels

# Any PNG or JPG image file should work
# Make sure to include the entire path with // instead of /

# e.g. your_test_image = "C://Users//vinitra.swamy//Pictures//digit.png"

your_test_image = "<path to file>"

import matplotlib.image as mpimg

if your_test_image != "<path to file>":
    img = mpimg.imread(your_test_image)
    plt.subplot(1,3,1)
    plt.imshow(img, cmap = plt.cm.Greys)
    print("Old Dimensions: ", img.shape)
    img = preprocess(img)
    print("New Dimensions: ", img.shape)
else:
    img = None

In [ ]:
if img is None:
    print("Add the path for your image data.")
else:
    input_data = json.dumps({'data': img.tolist()})

    try:
        r = json.loads(aci_service.run(input_data))
        result = r['result']
        time_ms = np.round(r['time'] * 1000, 2)
    except Exception as e:
        print(str(e), r['error'])

    plt.figure(figsize = (16, 6))
    plt.subplot(1, 15,1)
    plt.axhline('')
    plt.axvline('')
    plt.text(x = -100, y = -20, s = "Model prediction: ", fontsize = 14)
    plt.text(x = -100, y = -10, s = "Inference time: ", fontsize = 14)
    plt.text(x = 0, y = -20, s = str(result), fontsize = 14)
    plt.text(x = 0, y = -10, s = str(time_ms) + " ms", fontsize = 14)
    plt.text(x = -100, y = 14, s = "Input image: ", fontsize = 14)
    plt.imshow(img.reshape(28, 28), cmap = plt.cm.gray)    

In [ ]:
# remember to delete your service after you are done using it!
# uncomment the following line of code to delete your service

# aci_service.delete()

## Conclusion

Congratulations!

In this tutorial, you have managed to:
- familiarize yourself with the ONNX model format, ONNX Runtime inference, and the pretrained models in the ONNX model zoo
- understand a state-of-the-art convolutional neural net image classification model (MNIST in ONNX) and deploy it in the Azure ML cloud
- ensure that your deep learning model is working perfectly (in the cloud) on test data, and check it against some of your own!

Next steps:
- Check out another interesting application based on a Microsoft Research computer vision paper that lets you set up a [facial emotion recognition model](https://github.com/Azure/MachineLearningNotebooks/tree/master/onnx/onnx-inference-emotion-recognition.ipynb) in the cloud! This tutorial deploys a pre-trained ONNX Computer Vision model in an Azure ML virtual machine with GPU support.
- Contribute to our [open source ONNX repository on github](http://github.com/onnx/onnx) and/or add to our [ONNX model zoo](http://github.com/onnx/models)